In [1]:
import torch
import torch.nn as nn
from models.MMR import MMR
from models.MMR import TDB
from models.MMR import MMR_losses
# from models.Image2Recipe import Image2Recipe

# Define dummy data parameters
batch_size = 8  # Adjust as needed
seq_len = 10  # Length of the token sequence, adjust as needed
hidden_dim = 1024  # Size of the feature vectors
num_heads = 4  # Number of attention heads
ITEM_lyrs = 2  # Number of ITEM layers
MTD_lyrs = 2  # Number of MTD layers
projection_dim = 512  # Projection dimension



In [2]:
# Create dummy data (projected recipe and image embeddings)
self.image_encoder = Image_Encoder(self.device, self.clip_model, num_classes).to(self.device)
self.recipe_encoder = RecipeEncoder(self.device, self.vocab_size, self.max_len).to(self.device)
self.mmr = MMR(input_size=self.image_encoder.clip_model.config.projection_dim, device=self.device)
# num_heads=num_heads, ITEM_lyrs=ITEM_lyrs, MTD_lyrs=MTD_lyrs, projection_dim=projection_dim
self.model = Image2Recipe(self.image_encoder, self.recipe_encoder, self.mmr).to(self.device)


NameError: name 'Image_Encoder' is not defined

In [3]:
        
recipe_tokens = torch.randn(batch_size, seq_len, hidden_dim)  # Shape: [batch_size, seq_len, hidden_dim]
image_tokens = torch.randn(batch_size, seq_len, hidden_dim)  # Shape: [batch_size, seq_len, hidden_dim]
# Create labels for testing
labels = torch.randint(0, 2, (batch_size,)).float()  # Random binary labels for instance loss calculation


In [4]:
# Instantiate the MMR model
mmr_model = MMR(hidden_dim=hidden_dim, num_heads=num_heads, ITEM_lyrs=ITEM_lyrs, MTD_lyrs=MTD_lyrs, projection_dim=projection_dim)


In [5]:
print(recipe_tokens.shape)
print(image_tokens.shape)
print(labels.shape)

torch.Size([8, 10, 1024])
torch.Size([8, 10, 1024])
torch.Size([8])


In [6]:
# Forward pass through the model
logits = mmr_model(recipe_tokens, image_tokens)


self_attn
Query shape: torch.Size([8, 10, 1024]), Key shape: torch.Size([8, 10, 1024]), Value shape: torch.Size([8, 10, 1024])
cross_attn
Query shape: torch.Size([8, 10, 1024]), Key shape: torch.Size([8, 10, 1024]), Value shape: torch.Size([8, 10, 1024])
self_attn
Query shape: torch.Size([8, 10, 1024]), Key shape: torch.Size([8, 10, 1024]), Value shape: torch.Size([8, 10, 1024])
cross_attn
Query shape: torch.Size([8, 10, 1024]), Key shape: torch.Size([8, 10, 1024]), Value shape: torch.Size([8, 10, 1024])
self_attn
Query shape: torch.Size([8, 10, 1024]), Key shape: torch.Size([8, 10, 1024]), Value shape: torch.Size([8, 10, 1024])
cross_attn
Query shape: torch.Size([8, 10, 1024]), Key shape: torch.Size([8, 10, 1024]), Value shape: torch.Size([8, 10, 1024])
self_attn
Query shape: torch.Size([8, 10, 1024]), Key shape: torch.Size([8, 10, 1024]), Value shape: torch.Size([8, 10, 1024])
cross_attn
Query shape: torch.Size([8, 10, 1024]), Key shape: torch.Size([8, 10, 1024]), Value shape: torch.

In [7]:
print(logits.shape)

torch.Size([8])


In [8]:
print(labels)

tensor([0., 0., 0., 0., 0., 1., 0., 0.])


In [9]:
loss = MMR_losses(margin=1.0, instance_weight=1.0, sem_weight=1.0, itm_weight=1.0)
semantic_loss = loss.instance_semantic_loss(image_tokens, recipe_tokens, labels, margin=1, mode='semantic')
instance_loss = loss.instance_semantic_loss(image_tokens, recipe_tokens, labels, margin=1, mode='instance')
itm_loss = loss.itm_loss(logits, labels)

loss = loss.total_loss(labels, image_tokens, recipe_tokens, logits)

# Print the outputs for verification
print("Logits:", logits)
print("Instance Loss:", instance_loss.item())
print("Semantic Loss:", semantic_loss.item())

txt dist: torch.Size([8, 8])
posi mask: torch.Size([8, 8])
txt dist: torch.Size([8, 8])
posi mask: torch.Size([8, 8])
Logits: tensor([0.4274, 0.0142, 0.4427, 1.0000, 1.0000, 0.9960, 0.0229, 0.0028],
       grad_fn=<MeanBackward1>)
Instance Loss: 4.178627967834473
Semantic Loss: 2.4064154624938965


In [12]:
print(itm_loss)

tensor(4.1339, grad_fn=<BinaryCrossEntropyBackward0>)


In [10]:
print(loss)

tensor(10.7190, grad_fn=<AddBackward0>)
